## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим последовательный ансамбль решающих деревьев, используя облегченный градиентный бустинг (LightGBM). Используем перекрестную проверку, чтобы найти наилучшие параметры ансамбля.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных
Дополнительно преобразуем значение класса: начнем его с нуля.

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1

### Набор столбцов для расчета

In [4]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [5]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [6]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
36694 -0.395996 -1.200195  1.273438  0.324707 -1.634766 -0.169434  0.862305   
49283 -0.043610  0.269287  1.198242 -0.187134  0.611816 -0.169434  0.862305   
47506 -0.395996 -0.465576  0.667969 -0.177368 -1.634766 -0.169434  0.862305   
32456  0.802246  0.024353  1.878906  1.023438 -1.634766 -0.169434  0.862305   
20731 -1.100586 -0.955078 -0.315918 -0.846191  0.611816 -0.169434 -1.159180   

              7         8         9  ...       109       110       111  \
36694 -1.013672  0.862305 -0.928711  ... -0.083679  0.441650 -0.149292   
49283 -1.013672  0.862305 -0.928711  ... -0.083679  0.441650 -0.149292   
47506 -1.013672  0.864746 -0.928711  ... -0.083679  0.441650 -0.149292   
32456 -1.013672  0.862305 -0.928711  ... -0.083679  0.441650 -0.149292   
20731  1.100586 -1.156250  1.130859  ... -0.083679 -2.263672 -0.149292   

            112       113       114       115       116      117  Response  
366

### LightGBM
Основное отличие этого градиентного бустинга от предыдущих - использование сильно-разнородных (определяется разностью, гистограммой самих данных) экземпляров в выборке для формирования первоначального дерева: сначала рассматриваются все крайние, "плохие", случаи, а затем к ним "достраиваются" средние, "хорошие". Это позволяет еще быстрее минимизировать ошибку моделей.

Из дополнительных плюсов: алгоритм запускается сразу на всех ядрах процессора, это существенно ускоряет работу.

In [9]:
x = pd.DataFrame(data_train, columns=columns_transformed)
model = lgb.LGBMRegressor(random_state=17, max_depth=18,
                min_child_samples=19, num_leaves=34)

Также возможно провести классификации множества классов через LightGBM. В этом случае модель вернет вероятности принадлежности к каждому классу, возвращенные значения нужно будет дополнительно обработать через argmax, чтобы получить единственное значение класса.

In [ ]:
'''model = lgb.LGBMRegressor(random_state=17, max_depth=17,
                min_child_samples=18, num_leaves=34,
                objective="multiclass", num_class=8)'''

Диапазон тестирования параметров модели ограничен только вычислительной мощностью. Для проверки модели имеет смысл провести индивидуальные перекрестные проверки для каждого параметра в отдельности, затем в итоговой проверке перепроверить самые лучшие найденные параметры с отклонением +/-10%.

Проверку качества по каппа метрике при оптимизации выполнить не удастся из-за нецелых значений Light GBM. Гиперпараметры оптимизации:
* max_depth - максимальная глубина деревьев,
* num_leaves - число листьев в каждом
* min_child_samples - минимальное число элементов выборке в листе

In [10]:
lgb_params = {
    'max_depth': range(16,19),
    'num_leaves': range(34,37),
    'min_child_samples': range(17,20)
}
grid = GridSearchCV(model, lgb_params, cv=5, n_jobs=4, verbose=True)
grid.fit(x, data_train["Response"])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 135 out of 135 | elapsed: 13.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=18, min_child_samples=19,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=34,
                                     objective=None, random_state=17,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=4,
             param_grid={'max_depth': range(16, 19),
                         'min_child_samples': range(17, 20),
                         'num_leaves': range(34, 37)},
             pre

Выведем самые оптимальные параметры и построим итоговую модель, используя 1000 последовательных деревьев.

In [11]:
print (grid.best_params_)
model = lgb.LGBMRegressor(random_state=17,
    max_depth=grid.best_params_['max_depth'],
    min_child_samples=grid.best_params_['min_child_samples'],
    num_leaves=grid.best_params_['num_leaves'],
    n_estimators=1000,
    objective="multiclass", num_class=8)

{'max_depth': 18, 'min_child_samples': 17, 'num_leaves': 35}


In [12]:
model.fit(x, data_train["Response"])

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=18,
              min_child_samples=17, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_class=8, num_leaves=35,
              objective='multiclass', random_state=17, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

### Предсказание данных и оценка модели
LightGBM возвращает дробное значение класса, его нужно округлить.

Для multiclass используем argmax

In [13]:
def calculate_model (x):
    return np.argmax(model.predict([x]))

In [14]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target"] = x_test.apply(calculate_model, axis=1,
                                  result_type="expand")

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.536, градиентный бустинг - 0.56

In [15]:
print ("LightGBM:",
      round(cohen_kappa_score(data_test["target"],
                data_test["Response"], weights="quadratic"), 3))

LightGBM: 0.569


### Матрица неточностей

In [16]:
print ("LightGBM\n",
      confusion_matrix(data_test["target"], data_test["Response"]))

LightGBM
 [[ 326  158   25   14   41  107   49   34]
 [ 184  363   22    8  110  107   35   20]
 [  23   20   76   12    0    0    0    0]
 [  36   36   71  162    0    4    1    0]
 [  96  122    7    0  541   75    8    5]
 [ 231  249   14   23  222 1193  255  123]
 [ 103  141    3    6   70  314  671  176]
 [ 222  227    7   38   93  472  555 3571]]
